## Çoklu Doğrusal Regresyon

In [5]:
import pandas as pd
ad = pd.read_csv('Advertising.csv' , usecols = [1,2,3,4])
df = ad.copy()
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [9]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [11]:
X = df.drop("sales" , axis = 1)
y = df["sales"]

X_train, X_test , y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)


In [12]:
X_train.shape

(160, 3)

In [13]:
y_train.shape

(160,)

In [14]:
X_test.shape

(40, 3)

In [15]:
y_test.shape

(40,)

In [16]:
training = df.copy()

In [17]:
training.shape

(200, 4)

### Statsmodels

In [20]:
import statsmodels.api as sm

In [21]:
lm = sm.OLS(y_train, X_train)

In [23]:
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              2935.
Date:                Wed, 15 Mar 2023   Prob (F-statistic):                   1.28e-137
Time:                        17:48:18   Log-Likelihood:                         -336.65
No. Observations:                 160   AIC:                                      679.3
Df Residuals:                     157   BIC:                                      688.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0531      0.001     36.467      0.000       0.050       0.056
radio          0.2188      0.011     20.138      0.000       0.197       0.240
newspaper      0.0239      0.008      3.011      0.003       0.008       0.040
==============================================================================
Omnibus:                       11.405   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               15.574
Skew:                          -0.432   Prob(JB):                     0.000415
Kurtosis:                       4.261   Cond. No.                         13.5
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### R Squared Modelimizin Varyans Açıklama başarısını söyler.
### F-Statistic modelin anlamlı olup olmadığını belirtir.

# Scikit-Learn Model

In [27]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [29]:
lm = LinearRegression()
model = lm.fit(X_train, y_train)

In [31]:
sabit_katsayi = model.intercept_

In [32]:
print('Modelin Sabit Katsayısı: %.6f ' % (sabit_katsayi))

Modelin Sabit Katsayısı: 2.979067 


In [36]:
b_d_k = model.coef_

In [41]:
print("Bütün Bağımsız Değişkenlerin Katsayıları:" , (b_d_k))

Bütün Bağımsız Değişkenlerin Katsayıları: [0.04472952 0.18919505 0.00276111]


### Kurduğumuz Model ile bir Tahmin İşlemi Gerçekleştirelim.

#### Model Denklemi:
####    Sales = 2.97 + TV0.04 +; radio0.018 + newspaper*0.002
### 30 birim TV harcaması, 10 birim radio harcaması ve 40 birim de gazete harcaması olduğunda satışların tahmini değeri ne olur?

In [48]:
yeni_veri = [[30],[10],[40]]
yeni_veri = pd.DataFrame(yeni_veri).T

In [49]:
yeni_veri

,0,1,2
0,30,10,40


In [51]:
import warnings
warnings.filterwarnings('ignore')

In [56]:
olasi_ihtimal = model.predict(yeni_veri)

In [57]:
print('Olası İhtimal Dahilinde Satışların Tahmini Değeri: %.5f ' % (olasi_ihtimal))

Olası İhtimal Dahilinde Satışların Tahmini Değeri: 6.32335 


### Modelimizin Tahmin Değerini Kontrol Edelim.

In [59]:
from sklearn.metrics import mean_squared_error

In [63]:
rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
#model.predict(X_train) fonksiyonu bağımsız bir değişken olup model tahmin sayesinde tahmin edilen y değerlerini oluşturmamızı sağlar.

In [64]:
print('Eğitim Hatasının Değeri: %.10f ' % (rmse))

Eğitim Hatasının Değeri: 1.6447277656 


### Yukarıda Eğitim Hatamızı hesapladık şimdi ise Test Hatamızı Hesaplayalım.

In [66]:
rmse2 = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

In [67]:
print('Test Hatasının Değeri: %.10f ' % (rmse2))

Test Hatasının Değeri: 1.7815996615 


# Model Tuning / Model Doğrulama İşlemleri

#### 11.Satırdaki olan bu kodların içiinde olan random_state argümanı her değiştiğinde yeni değerler gelir.
#### Bunun önüne geçebilmek yani modelimiz için en uygun random_state argümanına vereceğimiz değişkeni seçmek için,
#### Cross Validation Score fonksiyonunu kullanabiliriz.

#### X= df.drop("sales" , axis = 1)
#### y = df["sales"]

#### X_train, X_test , y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)


In [74]:
#cross validasyon ile modelimizde random_state'i çaprazlayarak 10 farklı r**2 değeri oluşturuyoruz ve bunların ortalamasını alarak,
#bize en uygun valide edilmiş, doğrulanmış değeri seçiyoruz.
d_v_d = cross_val_score(model, X, y, cv = 10, scoring = "r2").mean()

In [75]:
print('Modelimizin Doğrulanmış (Valide Edilmiş) Değeri: %.10f ' % (d_v_d))

Modelimizin Doğrulanmış (Valide Edilmiş) Değeri: 0.8853562238 


### Bu işlemi bütün veri seti için yaptık şimdi ise train ve test setlerimize de uygulayalım.

In [83]:
d_v_d2 = cross_val_score(model, X_train, y_train, cv = 10, scoring = "r2").mean()

In [84]:
print('Modelimizin Eğitim Verileri İçindeki Doğrulanmış Validasyon Değeri: %.10f ' % (d_v_d2))

Modelimizin Eğitim Verileri İçindeki Doğrulanmış Validasyon Değeri: 0.7913548597 


In [94]:
train_val = np.sqrt(-cross_val_score(model,
                X_train, 
                y_train , 
                cv = 10 ,
                scoring = "neg_mean_squared_error")).mean()

In [96]:
print('Modelimizin Eğitim Verileri İçindeki Doğrulanmış R**2 Validasyon Değeri: %.10f ' % (train_val))

Modelimizin Eğitim Verileri İçindeki Doğrulanmış R**2 Validasyon Değeri: 1.6513523730 


### Yukarıda Eğitim Hatalarımızı Valide Ederek en güvenilir sonuca ulaşmış olduk.

### Aynı İşlemleri Test Seti Üzerinde de Gözlemleyelim

In [91]:
d_v_d3 = cross_val_score(model, X_test, y_test, cv = 10 , scoring = "r2").mean()

In [92]:
print('Modelimizin Test Verileri İçindeki Doğrulanmış Validasyon Değeri: %.10f ' % (d_v_d3))

Modelimizin Test Verileri İçindeki Doğrulanmış Validasyon Değeri: 0.7007204329 


In [97]:
test_val = np.sqrt(-cross_val_score(model,
                X_test, 
                y_test , 
                cv = 10 ,
                scoring = "neg_mean_squared_error")).mean()

In [98]:
print('Modelimizin Test Verileri İçindeki Doğrulanmış R**2 Validasyon Değeri: %.10f ' % (test_val))

Modelimizin Test Verileri İçindeki Doğrulanmış R**2 Validasyon Değeri: 1.8462778824 
